# EDA insurance data

In [0]:
# original data https://www.kaggle.com/c/allstate-purchase-prediction-challenge/data
# raw file from this repo: https://github.com/valx/ml_examples/raw/refs/heads/main/day1/train.csv.zip

In [0]:
table_name = "allstate_purchase"
fqn = f"main.insurance.{table_name}" 

In [0]:
%sql
USE CATALOG main;
USE SCHEMA insurance;

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/main/insurance/source/train.csv")

In [0]:
display(df)

In [0]:
# df.count() - 665249
df.select('customer_ID', 'shopping_pt').distinct().count()

In [0]:
display(df.describe())

In [0]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)

In [0]:
%sql
SELECT * FROM allstate_purchase

In [0]:
import seaborn as sns

In [0]:
df = spark.table(table_name)

In [0]:
print(df.columns)

cols = [ 
 'group_size',
 'homeowner',
 'car_age',
 'car_value',
 'risk_factor',
 'age_oldest',
 'age_youngest',
 'married_couple',
 'duration_previous',
 'cost']

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

df_pd = df.select(cols).sample(0.001).toPandas()
sns.set(style="whitegrid")
plt.figure(figsize=(6, 4))
sns.pairplot(df_pd, kind="reg")
plt.tight_layout()
plt.show()

# Training set

In [0]:
# databricks-sdk==0.50.0 databricks-automl-runtime==0.2.21 holidays==0.71 category-encoders==2.8.1 lightgbm==4.6.0

In [0]:
%pip install mlflow==2.22.0 databricks-feature-engineering==0.10.2 
dbutils.library.restartPython()

In [0]:
from pyspark.sql import functions as F
from databricks.feature_engineering import FeatureEngineeringClient

fe = FeatureEngineeringClient()

# example https://docs.databricks.com/aws/en/notebooks/source/machine-learning/feature-store-with-uc-basic-example.html

In [0]:
table_name = "allstate_purchase"
fqn = f"main.insurance.{table_name}"
fqn

In [0]:
df = spark.table(table_name)

df = df.withColumn('id', F.concat(F.col('customer_ID'), F.col('shopping_pt'))).drop("customer_ID", "shopping_pt")


In [0]:
df.schema

In [0]:
%sql
-- DROP TABLE allstate_purchase_fe

-- ALTER TABLE main.insurance.allstate_purchase_fe ALTER COLUMN id SET NOT NULL;
-- ALTER TABLE main.insurance.allstate_purchase_fe ADD CONSTRAINT table_name_pk PRIMARY KEY( id );

In [0]:
fqn+'_fe'

In [0]:
customer_feature_table = fe.create_table(
  name=fqn+'_fe',
  primary_keys='id',
  #df=df,
  schema=df.drop('cost').schema,
  description='Customer features'
)

In [0]:
%sql
SELECT * FROM main.insurance.allstate_purchase_fe

In [0]:
df.drop('cost').write.mode("append").saveAsTable(table_name+'_fe')
#df.write.mode("overwrite").option("overwriteSchema", "true").option("primaryKey", "id").saveAsTable(table_name+'_fe')

In [0]:
df.select('id','cost').write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name+'_main')
df_main = spark.table(table_name+'_main')

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup

model_feature_lookups = [FeatureLookup(table_name=fqn+'_fe', lookup_key='id')]

# fe.create_training_set looks up features in model_feature_lookups that match the primary key from inference_data_df
training_set = fe.create_training_set(df=df_main, feature_lookups=model_feature_lookups, label="cost")


In [0]:
display(training_set.load_df())

In [0]:
user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

user

In [0]:
import mlflow # Storing artifacts in a volume requires MLflow 2.15.0 or above

EXP_NAME = f"/Users/{user}/my_experiment_name"
ARTIFACT_PATH = f"dbfs:/Volumes/main/insurance/source" # can be a managed or external volume

mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")

if mlflow.get_experiment_by_name(EXP_NAME) is None:
    mlflow.create_experiment(name=EXP_NAME, artifact_location=ARTIFACT_PATH)
mlflow.set_experiment(EXP_NAME)


In [0]:
%pip install xgboost 

In [0]:
# python list comprehension - columns to be used as features
feature_cols = [c[0] for c in df.dtypes if c[1] in ['int', 'double'] and c[0] not in ['id', 'cost']]
print(feature_cols)

In [0]:
import xgboost as xgb


import mlflow.xgboost
from sklearn.model_selection import train_test_split

mlflow.xgboost.autolog()

df_pd = training_set.load_df().toPandas()

X = df_pd[feature_cols]
y = df_pd['cost']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist"
}

with mlflow.start_run():
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dtest, "test")],
        early_stopping_rounds=10
    )

In [0]:
display(y_test) #X_train, X_test, y_train, y_test


In [0]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(xgb.DMatrix(X_test))
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

display({"RMSE": rmse, "R2": r2})

In [0]:
import mlflow
import mlflow.xgboost
from mlflow.models.signature import infer_signature

signature = infer_signature(X_train, y_train)

model_uri = f"models:/main.insurance/xgboost_model"
mlflow.end_run()
with mlflow.start_run():
    mlflow.xgboost.log_model(
        model,
        artifact_path="xgboost_model",
        signature=signature,
        registered_model_name="main.insurance.xgboost_model"
    )

In [0]:
import mlflow.xgboost

model_uri = "models:/main.insurance.xgboost_model/1"
loaded_model = mlflow.xgboost.load_model(model_uri)

y_pred = loaded_model.predict(dtest)

In [0]:
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


display({"RMSE": rmse, "R2": r2})